## 概要
* 漢字をカタカナに変換
* MeCabでカタカナに変換できなかったものは除外
* 季語がない無季はとりあえず除外

In [13]:
import MeCab
import pandas as pd
import re
import jaconv
import numpy as np

In [2]:
#tagger = MeCab.Tagger("-Ochasen -d /usr/local/lib/mecab/dic/mecab-ipadic-neologd")
tagger = MeCab.Tagger("-Ochasen")

In [5]:
new=pd.read_csv("df_cleaning3.csv")

In [7]:
new["kigo"].nunique()

4620

In [39]:
new[new["haiku"].apply(lambda x: "枯蓮" in x)]

,Unnamed: 0,cd-num,haiku,writer,kigo,season,yomi,kigo_yomi
1150,1150,1231,枯蓮の折れたる影は折れてをる,富安風生,枯蓮,冬,枯蓮ノオレタルカゲハオレテヲル,枯蓮
2035,2035,2159,枯蓮のうごく時きてみなうごく,西東三鬼,枯蓮,冬,枯蓮ノウゴクトキキテミナウゴク,枯蓮
2330,2330,2469,枯蓮に隈おとしたる道化たち,橋閒石,枯蓮,冬,枯蓮ニクマオトシタルドウケタチ,枯蓮
6263,6263,7518,枯蓮や無用の者ら無用の首,井沢唯夫,枯蓮,冬,枯蓮ヤムヨウノモノラムヨウノクビ,枯蓮
12015,12015,14283,裏返る力もうなし枯蓮,畠中定子,枯蓮,冬,ウラガエルリキモウナシ枯蓮,枯蓮
12597,12597,14872,杖ついて水の枯蓮歳をとる,竹鼻瑠璃男,枯蓮,冬,ツエツイテミズノ枯蓮トシヲトル,枯蓮
16497,16497,18794,枯蓮己の影の怖からむ,藤岡尚子,枯蓮,冬,枯蓮オノレノカゲノコワカラム,枯蓮
17427,17427,19738,筑波嶺を背に枯蓮枯蓮,菅原けい,枯蓮,冬,ツクバミネヲセニ枯蓮枯蓮,枯蓮
18625,18625,20942,枯蓮に柔らかき水ありにけり,原田宏子,枯蓮,冬,枯蓮ニヤワラカキミズアリニケリ,枯蓮
19440,19440,21764,一望の枯蓮に火の色を見き,江中真弓,枯蓮,冬,イチボウノ枯蓮ニヒノイロヲミキ,枯蓮


In [40]:
def convert(text):
    sentence = []
    if text[-1]=="哉":
        text=text[:-1]+"かな"
    if "枯葎" in text:
        text = re.sub(r"枯葎", "カレムグラ", text)
    if "夏瘦" in text:
        text = re.sub(r"夏瘦", "ナツヤセ", text)
    if "枯薄" in text:
        text = re.sub(r"枯薄", "カレススキ", text)
    if "蟬" in text:
        text = re.sub(r"蟬", "セミ", text)
    if "魂迎" in text:
        text = re.sub(r"魂迎", "タマムカエ", text)
    if "枯蓮" in text:
        text= re.sub(r"枯蓮", "カレハス", text)
    node = tagger.parse(text)
    node = node.split("\n")
    for i in range(len(node)):
        feature = node[i].split("\t")
        if feature[0] == "EOS":
            break
        sentence.append(feature[1])
        #print(feature)
    return jaconv.hira2kata("".join(sentence))

In [41]:
new["yomi"]=new["haiku"].apply(convert)

In [42]:
new["kigo_yomi"]=new["kigo"].apply(convert)

In [44]:
list("あいうおえ")

['あ', 'い', 'う', 'お', 'え']

In [47]:
x = list(map(lambda x: list(x) ,new["yomi"].values))

In [51]:
vocab=np.unique(np.concatenate(x, axis=0))

In [31]:
new.shape

(38188, 8)

In [4]:
def is_kana(text):
    if re.fullmatch(r'[ア-ン]*', text):
        return True
    else:
        return False

In [61]:
only_kana = new[new["yomi"].apply(is_kana)]

In [64]:
only_kana.shape

(32475, 8)

In [65]:
only_kana2 = only_kana[only_kana["kigo"]!="無季"]

In [69]:
only_kana2[["yomi", "kigo_yomi"]].to_csv("../save/only_kana2.csv", index=False)

In [70]:
only_kana2.shape

(31623, 8)

In [71]:
only_kana2["kigo_yomi"].nunique()

3985

In [72]:
only_kana2["kigo_yomi"].unique()

array(['キリ', 'カレムグラ', 'ナノハナ', ..., 'ホッキガイ', 'ショシ', 'トシトクジン'], dtype=object)

In [2]:
df = pd.read_csv("../save/only_kana2.csv")

In [5]:
df.shape

(31623, 2)

In [10]:
df2 = df[df["kigo_yomi"].apply(is_kana)]

In [15]:
df2.to_csv("../save/only_kana2.csv", index=False)